# Dados de Galton: estudo sobre Alturas 

<br>
<center>
<img src="https://media.giphy.com/media/3o7btPCcdNniyf0ArS/giphy.gif" alt="drawing" width="500px"></center>

In [ ]:
#-- carregando as libs
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
#-- importando os dados
galton = pd.read_csv('../data/Galton.csv')
galton.head()

#-- dicionário dos dados
 - **Family:** Família de origem
 - **Father:** Altura do Pai
 - **Mother:** Altura da Mãe
 - **Gender:** Sexo do Filho
 - **Height:** Altura do Filho
 - **Kids:** Quantidade de Filhos

In [ ]:
#-- verificando as dimensões do dataset
galton.shape

In [ ]:
#-- verificando os tipos das variáveis
galton.info()

### Análise Exploratória dos Dados

 - Análise exploratória serve para desvendar um pouco do processo gerador de dados. Quanto mais você souber sobre como os dados são gerados, melhor será suas chances de usar a técnica de modelagem mais adequada. Algumas análises são padrões em todos os processos de análise exploratória:
1. Análise de distribuição. Ex: qual a média, mínimo e máximo de cada variável
2. Análise de dados faltantes. Ex: Quais variáveis têm dados faltantes? Qual a proporção de dados faltantes por variável? Porque esses dados estão faltando? É um bug ou uma característica do processo gerador de dados?
3. Variáveis categóricas. Existem variáveis categóricas? Qual a cardinalidade das variáveis categóricas? As categorias são ordenadas ou sem ordem?
4. Correlação. Qual a correlação entre variáveis?

In [ ]:
#-- verificando a distribuição dos dados
galton.describe().round(4)

In [ ]:
#-- verificando os missing values
galton.isnull().sum()

In [ ]:
#-- verificando a cardinalidade dos dados
galton.apply(pd.Series.nunique)

In [ ]:
#-- correlação
_ = sns.heatmap(galton.corr())

> Correlação acima de 0,70 entre as variáveis independentes 

In [ ]:
#-- importando scatter matrix
from pandas.plotting import scatter_matrix

continuous_cols = galton.select_dtypes(include=['float']) #-- selecionando os dados numéricos
scatter_matrix(continuous_cols, figsize=(15, 10))
plt.show()

## Parte_1: Ajustando um Modelo de regressão linear simples para determinar a relação entre a Altura do Pai e a Altura do Filho
## Sklearn

In [ ]:
Y = galton['Height']
X = galton['Father'].to_frame()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, Y_train)

Y_pred = lm.predict(X_test)

In [ ]:
_ = sns.regplot(x=Y_test, y=Y_pred)

In [ ]:
beta1=lm.coef_
intercepto=lm.intercept_
print(beta1)
print(intercepto)

In [ ]:
X_test.iloc[0]

In [ ]:
Y_pred[0]

In [ ]:
intercepto + (beta1[0] * X_test.iloc[0][0])

In [ ]:
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred)
print(mse)

**MSE**

$\frac{1}{N}\sum{(y_{test} - y_{pred})^2}$

O MSE calcula o erro quadrático médio das predições do nosso modelo. Quanto maior o MSE, pior é o modelo.

Essa métrica é útil quando temos valores de target mais incomuns e que seriam interessantes que nosso modelo não errasse.

Por outro lado, como os erros são penalizados exponencialmente, erros maiores tem um peso maior do que os erros menores. Então, se o nosso modelo faz apenas uma predição muito, muito ruim, o MSE irá elevar esse erro ao quadrado e, com isso, esse erro ficará ainda pior e acabaremos achando que o nosso modelo está performando pior do que realmente está.

Outro ponto é que a escala do MSE não é a mesma do nosso target (visto que os erros são elevados ao quadrado), então sua interpretação fica mais difícil.

In [ ]:
#Calculando o R^2
sklearn.metrics.r2_score(Y_test, Y_pred)

**R²**

$1 - \frac{\frac{1}{N}\sum{(y_{test} - y_{pred})^2}}{\frac{1}{N}\sum{(y_{test} - \bar{y})^2}}$

O R² é uma métrica que varia entre -∞ e 1 e é uma razão que indica o quão bom o nosso modelo está em comparação com um modelo naive que faz a predição com base no valor médio do target. Quanto maior seu valor, melhor é nosso modelo com relação a esse modelo mais simplista.

Um valor de R² negativo significa que nosso modelo é pior do que se tivéssemos feito a predição com o valor da média. Entretanto, só com essa métrica não conseguimos enxergar a magnitude dos erros do nosso modelo.

## Parte_2: Ajustando um Modelo de regressão linear simples para determinar a relação entre a Altura do Pai e a Altura do Filho.
## Statsmodel

In [ ]:
#Ajustando Mod de reg linear na biblioteca statsmodel
#Usando a biblioteca statsmodels
import statsmodels.api as sm

X_7 = sm.add_constant(X) #adiciona a cte a base de dados
est7 = sm.OLS(Y, X_7)
est77 = est7.fit()
print(est77.summary())

In [ ]:
Y_pred = est77.predict(X_7)

In [ ]:
mse = sklearn.metrics.mean_squared_error(Y, Y_pred)
print(mse)

In [ ]:
#Calculando o R^2
sklearn.metrics.r2_score(Y, Y_pred)

## Parte_3: Ajustando um Modelo de Regressão Linear Multipla usando statsmodel
Utilizando as Variáveis: altura do pai, da mãe e quantidade de filhos

In [ ]:
#-- zerando os datasets
Y = []
X = []

In [ ]:
Y = galton['Height']
X = galton[['Father', 'Mother', 'Kids']]

In [ ]:
#Usando a biblioteca statsmodels
#Modelo COM intercepto
X2 = sm.add_constant(X) #adiciona a cte a base de dados
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

## Parte_4: Criando uma nova variável
Utilizando as Variáveis: altura do pai, da mãe e a média da altura dos pais

In [ ]:
#Cria uma nova variável que é a média das alturas dos pais
galton["Mean_Parents"]= (galton['Father'] + galton['Mother'])/2

In [ ]:
#-- zerando os datasets
Y = []
X = []

In [ ]:
Y = galton['Height']
X = galton[['Father', 'Mother', 'Mean_Parents']]

In [ ]:
#Usando a biblioteca statsmodels
#Modelo COM intercepto
X3 = sm.add_constant(X) #adiciona a cte a base de dados
est = sm.OLS(Y, X3)
est2 = est.fit()
print(est2.summary())

## Parte_5: Ajustando um Modelo de regressão linear simples para determinar a relação entre a Altura do Pai e a Altura do Filho
## Sklearn

In [ ]:
Y = []
X = []

In [ ]:
Y = galton['Height']
X = galton[['Father', 'Mother', 'Mean_Parents']]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, Y_train)

Y_pred = lm.predict(X_test)

In [ ]:
_ = sns.regplot(x=Y_test, y=Y_pred)

In [ ]:
beta1=lm.coef_
intercepto=lm.intercept_
print(beta1)
print(intercepto)

In [ ]:
X_test.iloc[0]

In [ ]:
Y_pred[0]

In [ ]:
intercepto + (beta1[0] * X_test.iloc[0][0]) + (beta1[1] * X_test.iloc[0][1]) + (beta1[2] * X_test.iloc[0][2])

In [ ]:
mse = sklearn.metrics.mean_squared_error(Y_test, Y_pred)
print(mse)

In [ ]:
#Calculando o R^2
sklearn.metrics.r2_score(Y_test, Y_pred)